In [11]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

In [12]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [2]:
import minsearch
import json
from openai import OpenAI

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [7]:
q = 'the course has already started, can I still enroll?'

In [33]:
def search(query):
    boost = {'question':2.5, 'section':0.5}

    results = index.search(
        query=query, 
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost, 
        num_results=10
    )
    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
    """.strip()

    context = ""
        
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role":"user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [34]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [36]:
query = 'how do i run kafka?'
answer = rag(query)

"To run Kafka, especially when dealing with a Python setup, it is recommended to create a virtual environment and manage dependencies within that environment. Here's how you can do it:\n\n1. **Create a Virtual Environment:**\n   - Run this command to create a virtual environment:\n     ```\n     python -m venv env\n     ```\n   - Activate the environment:\n     - On MacOS or Linux:\n       ```\n       source env/bin/activate\n       ```\n     - On Windows:\n       ```\n       env\\Scripts\\activate\n       ```\n\n2. **Install Required Packages:**\n   - Install the necessary packages as specified in `requirements.txt`:\n     ```\n     pip install -r ../requirements.txt\n     ```\n\n3. **Ensure Docker is Running:**\n   - It's important that your Kafka broker Docker container is running.\n   - Validate by running:\n     ```\n     docker ps\n     ```\n   - If it's not running, navigate to your docker compose yaml file folder and start all instances with:\n     ```\n     docker-compose up -

In [37]:
answer

"To run Kafka, especially when dealing with a Python setup, it is recommended to create a virtual environment and manage dependencies within that environment. Here's how you can do it:\n\n1. **Create a Virtual Environment:**\n   - Run this command to create a virtual environment:\n     ```\n     python -m venv env\n     ```\n   - Activate the environment:\n     - On MacOS or Linux:\n       ```\n       source env/bin/activate\n       ```\n     - On Windows:\n       ```\n       env\\Scripts\\activate\n       ```\n\n2. **Install Required Packages:**\n   - Install the necessary packages as specified in `requirements.txt`:\n     ```\n     pip install -r ../requirements.txt\n     ```\n\n3. **Ensure Docker is Running:**\n   - It's important that your Kafka broker Docker container is running.\n   - Validate by running:\n     ```\n     docker ps\n     ```\n   - If it's not running, navigate to your docker compose yaml file folder and start all instances with:\n     ```\n     docker-compose up -

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [11]:
es_client.info()

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Content-Type, Accept]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)